In [1]:
from __future__ import print_function
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [46]:
class CurrencyDataset(Dataset):
    def __init__(self,root_dir):
        self.dir = os.path.abspath(root_dir)
    def __getitem__(self,idx):
        data_path = os.path.join(self.dir,'Batches','Batch_' + str(idx) + '.npy')
        label_path = os.path.join(self.dir,'Labels','Label_' + str(idx) + '.npy')
        data = np.load(data_path)
        labels = np.load(label_path)
        return (data,labels)
    def __len__(self):
        return len(os.listdir(os.path.join(self.dir,'Batches')))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,1,(1,50))
        self.fc1 = nn.Linear(1, 3000)
        self.dp1 = nn.Dropout(p=0.9)
        self.fc2 = nn.Linear(3000, 3500)
        self.fc3 = nn.Linear(3500, 1)

    def forward(self, x):
        print(x.shape)
     #   x = self.dp1(x)
#         print(x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.fc1(x))
        print(x.shape)
        x = self.dp1(x)
        x = F.relu(self.fc2(x))
        x = self.dp1(x)
        x = self.fc3(x)
        print(x.shape)
        x = x.view(-1,2)
        x = F.softmax(x,dim=1)
        return x
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        print('Success')
        torch.nn.init.xavier_uniform_(m.weight)

In [47]:
torch.set_default_tensor_type('torch.DoubleTensor')
data = CurrencyDataset('./Processed')
net = Net()

net = net.double()
net = net.cuda()
net.train()
dataloader = DataLoader(data,batch_size=2000,shuffle=True,pin_memory=True)

In [48]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [49]:
running_loss = 0.0
net.apply(weight_reset)
for i in range(2):
    for i,batch in enumerate(dataloader):
        optimizer.zero_grad()
        d,y = batch
        
        d = d.cuda()
        x = net(d)
        y = (y.reshape([-1,2])).cuda()
        z = torch.mul(x,y)
        z = z.sum(dim=1)
        loss = z.sum()
        loss /= z.shape[0]
        loss = loss ** -1
        loss.backward()
        optimizer.step()

Success
Success
Success
Success
torch.Size([2000, 3, 2, 50])
torch.Size([2000, 1, 2, 3000])
torch.Size([2000, 1, 2, 1])
torch.Size([2000, 3, 2, 50])
torch.Size([2000, 1, 2, 3000])
torch.Size([2000, 1, 2, 1])


In [ ]:
net.eval()
for i,batch in enumerate(dataloader):
    d,y = batch
    print(y)
    print(net(d.cuda()))
    break